In [1]:
import io
import re
import zipfile
import urllib.request
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import MeCab
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
from gensim import corpora, matutils

In [2]:
# 青空文庫の「吾輩は猫である」を読み込む
with urllib.request.urlopen('https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip') as r:
    # zipファイルをバイト型で読み込む
    data = r.read()
    with zipfile.ZipFile(io.BytesIO(data), 'r') as zipdata:
        with zipdata.open(zipdata.namelist()[0], 'r') as f:
            text = f.read()
            # shift-jisでデコード
            text = text.decode('shift_jis')

# '-----'で区切られた部分を分割し、本文が始まる部分を取得。non-greedyマッチングは?をつける
text = re.split(r'\-{55}', text)[2]

# ルビを削除
text = re.sub(r'《.+?》', '', text)

# ルビの付く文字列の始まりを示す記号を削除
text = re.sub(r'｜', '', text)

# 入力者注を削除
text = re.sub(r'［＃.+?］', '', text)

# アクセント分解された欧文を囲む記号を削除
text = re.sub(r'〔.+?〕', '', text)

# '底本；'で分割し、最初の部分を取得（必要に応じて）
text = text.split('底本；')[0]

# 最終的なテキストを単語に分割
words = text.split()

# 空白文字を削除
text = text.replace('\u3000','')
# 改行コードを削除
text = text.replace('\r','').replace('\n', '')
# 「。」を区切り文字として分割
sentences = text.split('。')
print(sentences[:5])

['一吾輩は猫である', '名前はまだ無い', 'どこで生れたかとんと見当がつかぬ', '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している', '吾輩はここで始めて人間というものを見た']


In [3]:
words_list = []
# 各文に形態素解析を行う
t = MeCab.Tagger('-Ochasen')
# 各文に対して処理、最後は文字列がないため省略
for sentence in sentences[:-1]:
    sentence_parsed = t.parse(sentence)
    word_s = []
    # 各文に現れる単語のリストに対して処理を往復
    for line in sentence_parsed.splitlines()[:-1]:
        word_s.append(line.split('\t')[2])
    words_list.append(word_s)


In [4]:
words_list[1]

['名前', 'は', 'まだ', '無い']

In [32]:
# 日本語極性辞書を読み込む
with urllib.request.urlopen('https://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn') as f:
    text_wago = f.read().decode('utf-8')

# dfとして読み込む
df = pd.read_csv(io.StringIO(text_wago), header=None, sep='\t')

# 単語とスコアを対応させる辞書を作成
word2score = {}
values = {
    'ポジ（経験）':1,
    'ポジ（評価）':1,
    'ネガ（経験）':-1,
    'ネガ（評価）':-1
}

for word, label in zip(df.loc[:,1], df.loc[:,0]):
    word2score[word] = values[label]

scores = []
# 各文書のスコアを算出
for words in words_list:
    score = 0
    # 単語が辞書に現れていればそのスコアを加算
    for word in words:
        if word in word2score:
            score += word2score[word]
    scores.append(score)

scores_df = pd.DataFrame({'sentence': sentences[:-1],
                          'scores': scores}, columns=['sentence', 'scores'])
scores_df.head()

,sentence,scores
0,一吾輩は猫である,0
1,名前はまだ無い,0
2,どこで生れたかとんと見当がつかぬ,0
3,何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している,-1
4,吾輩はここで始めて人間というものを見た,0


,sentence,score
count,7489,0
unique,7486,0
top,「あのね,NaN
freq,2,NaN
